# Data prep


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd

In [46]:
gdf_old = gpd.read_file("C:/Users/Lara/AWI/Old_AWI_Wilts.shp")
gdf_new = gpd.read_file("C:/Users/Lara/AWI/New_AWI.shp")
gdf_new_reduced = gdf_new[['NAME_OS', 'NAME_EP1', 'P3_UID', 'P3_IGNORE', 'C21_AP', 'EPOCH_2',
       'EPOCH_1', 'C20_AP', 'C20_AP_REF', 'TITHE_REF', 'C19_EM', 'C19_EM_REF',
       'OSD', 'OSD_REF', 'CM_1773', 'C18_EM', 'C18_EM_REF', 'C17_EM',
       'C17_EM_REF', 'EM', 'EM_REF', 'OTHER', 'OTHER_REF', 'NAME_TITHE',
       'NAME_OSD', 'NAME_EM', 'TITHE_REF_', 'TITHE_LAND', 'TITHE_PLOT',
       'TITHE_PL_1', 'COMMENTS', 'url', 'OSNAMES_UR', 'CHECK_NAME',
       'TO_CHECK_N', 'NOTES', 'LISTED_BUI', 'LISTED_PAR', 'P2_CLASS',
       'P3_CLASS', 'P3_Status', 'TITHE', 'GlobalID_1', 'EARLIEST_E', 'RESERVE',
       'FORESTRY', 'SURVEY_PRI', 'ACTION', 'SITE_ID', 'AreaHa', 'NAME',
       'Shape_Leng', 'OBJECTID_1', 'HE_REF', 'geometry']]

gdf_new_reduced['intersects_old_AWI'] = gdf_new_reduced.geometry.apply(lambda x: gdf_old.geometry.intersects(x).any())


C:\Users\Lara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [47]:
#df = pd.read_excel('O:/NaturalEngland/Ancient Woodland Inventory Update/Analysis/AWI_data_analysis_assessment.xlsm', sheet_name = 'AWI_data_RAW')
df = gdf_new_reduced.copy()

In [48]:
df.columns

Index(['NAME_OS', 'NAME_EP1', 'P3_UID', 'P3_IGNORE', 'C21_AP', 'EPOCH_2',
       'EPOCH_1', 'C20_AP', 'C20_AP_REF', 'TITHE_REF', 'C19_EM', 'C19_EM_REF',
       'OSD', 'OSD_REF', 'CM_1773', 'C18_EM', 'C18_EM_REF', 'C17_EM',
       'C17_EM_REF', 'EM', 'EM_REF', 'OTHER', 'OTHER_REF', 'NAME_TITHE',
       'NAME_OSD', 'NAME_EM', 'TITHE_REF_', 'TITHE_LAND', 'TITHE_PLOT',
       'TITHE_PL_1', 'COMMENTS', 'url', 'OSNAMES_UR', 'CHECK_NAME',
       'TO_CHECK_N', 'NOTES', 'LISTED_BUI', 'LISTED_PAR', 'P2_CLASS',
       'P3_CLASS', 'P3_Status', 'TITHE', 'GlobalID_1', 'EARLIEST_E', 'RESERVE',
       'FORESTRY', 'SURVEY_PRI', 'ACTION', 'SITE_ID', 'AreaHa', 'NAME',
       'Shape_Leng', 'OBJECTID_1', 'HE_REF', 'geometry', 'intersects_old_AWI'],
      dtype='object')

In [49]:
# Defining functions

def check_if_present(FIELD):
    if FIELD == 1 or FIELD ==2 or FIELD ==6:
        return 1
    else:
        return 0
    
def check_if_zero(FIELD):
    if FIELD == 0:
        return 1
    else:
        return 0
    
def check_if_name(FIELD):
    if FIELD.strip() == '':
        return 0
    else:
        return 1


In [50]:
# Columns to check if if parcel has been marked as woodland
columns_to_assess = ['TITHE','C19_EM', 'OSD', 'CM_1773', 'C18_EM', 'C17_EM', 'EM','OTHER', 'HE_REF' ]

#Loop through each parcel and create new column showing if woodland present in source, and if source is zero (not woodland)
for i in columns_to_assess:
    df[f"present_{i}"] = df[i].apply(check_if_present)
    df[f"zero_{i}"] = df[i].apply(check_if_zero)

In [51]:
# Combine columns showing woodland present to see how many sources show woodland present
df['present_combined'] = df['present_TITHE']+df['present_C19_EM'] +df['present_OSD']+df['present_CM_1773'] +df['present_C18_EM']+df['present_C17_EM'] +df['present_EM']+df['present_OTHER'] +df['present_HE_REF']
df['present_primary'] = df['present_TITHE'] + df['present_OSD'] + df['present_C17_EM']+df['present_HE_REF']
df['present_secondary'] =df['present_C19_EM']+df['present_CM_1773'] +df['present_C18_EM'] +df['present_EM']+df['present_OTHER'] 

# Combine columns showing if zero present to show how many sources show no woodland present
df['zero_combined'] = df['zero_TITHE']+df['zero_C19_EM'] +df['zero_OSD']+df['zero_CM_1773'] +df['zero_C18_EM']+df['zero_C17_EM'] +df['zero_EM']+df['zero_OTHER'] +df['zero_HE_REF']
df['zero_primary'] = df['zero_TITHE']+df['zero_OSD']
df['zero_secondary'] =  df['zero_C19_EM'] +df['zero_CM_1773'] +df['zero_C18_EM']+df['zero_C17_EM'] +df['zero_EM']+df['zero_OTHER'] +df['zero_HE_REF']


In [52]:
df.columns

Index(['NAME_OS', 'NAME_EP1', 'P3_UID', 'P3_IGNORE', 'C21_AP', 'EPOCH_2',
       'EPOCH_1', 'C20_AP', 'C20_AP_REF', 'TITHE_REF', 'C19_EM', 'C19_EM_REF',
       'OSD', 'OSD_REF', 'CM_1773', 'C18_EM', 'C18_EM_REF', 'C17_EM',
       'C17_EM_REF', 'EM', 'EM_REF', 'OTHER', 'OTHER_REF', 'NAME_TITHE',
       'NAME_OSD', 'NAME_EM', 'TITHE_REF_', 'TITHE_LAND', 'TITHE_PLOT',
       'TITHE_PL_1', 'COMMENTS', 'url', 'OSNAMES_UR', 'CHECK_NAME',
       'TO_CHECK_N', 'NOTES', 'LISTED_BUI', 'LISTED_PAR', 'P2_CLASS',
       'P3_CLASS', 'P3_Status', 'TITHE', 'GlobalID_1', 'EARLIEST_E', 'RESERVE',
       'FORESTRY', 'SURVEY_PRI', 'ACTION', 'SITE_ID', 'AreaHa', 'NAME',
       'Shape_Leng', 'OBJECTID_1', 'HE_REF', 'geometry', 'intersects_old_AWI',
       'present_TITHE', 'zero_TITHE', 'present_C19_EM', 'zero_C19_EM',
       'present_OSD', 'zero_OSD', 'present_CM_1773', 'zero_CM_1773',
       'present_C18_EM', 'zero_C18_EM', 'present_C17_EM', 'zero_C17_EM',
       'present_EM', 'zero_EM', 'present_OTHER', '

In [53]:
# Check which parcels are named

df['NAME_TITHE'] = df['NAME_TITHE'].astype(str)

names_to_assess = ['NAME_TITHE','NAME_EP1', 'NAME_OSD', 'NAME_EM']
# Add column showing which parcels are named in source
for i in names_to_assess:
    df[i] = df[i].astype(str)
    df[f"present_{i}"] = df[i].apply(check_if_name)

# Combine to get total named sources for parcel    
df['name_present'] = df['present_NAME_EM']+df['present_NAME_OSD']+df['present_NAME_EP1'] +df['present_NAME_TITHE']

# Decision tree


In [54]:
# Filter for primary source present only
df_primary = df[df.present_primary >=1]

## A - Ancient woodland
## Contains primary evidence and no zero's present
df_primary_no_zero = df_primary[df_primary.zero_combined ==0]
df_primary_no_zero['Category'] = 'Ancient woodland'
df_primary_no_zero['subCategory'] = 'A'

#Filter for primary source present, one zero present
df_primary_one_zero = df_primary[df_primary.zero_combined ==1]

## A - Not ancient woodland 
## Has primary evidence but contains more than one zero
df_primary_plural_zero = df_primary[df_primary.zero_combined >1]
df_primary_plural_zero['Category'] = 'Not ancient woodland'
df_primary_plural_zero['subCategory'] = 'A'

## B - Ancient woodland
## Has primary evidence, one zero present, but is a named woodland 
df_primary_one_zero_named = df_primary_one_zero[df_primary_one_zero.name_present >=1]
df_primary_one_zero_named['Category'] = 'Ancient woodland'
df_primary_one_zero_named['subCategory'] = 'B'

## A- Maybe ancient woodland
## Has primary evidence, one zero present and is not a named woodland. 
### Further research needed, what source is zero?
df_primary_one_zero_not_named = df_primary_one_zero[df_primary_one_zero.name_present==0]
df_primary_one_zero_not_named['Category'] = 'Maybe ancient woodland'
df_primary_one_zero_not_named['subCategory'] = 'A'


C:\Users\Lara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Lara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Lara\Ap

In [55]:
# Filter for no primary evidence present, but secondary evidence present
df_secondary = df[(df.present_primary ==0)&(df.present_secondary >=1)]
# Filter for no evidence present
df_no_evidence = df[(df.present_primary ==0)&(df.present_secondary ==0)]

## D - Maybe ancient woodland
## No evidence, but contains no zeros, so may just not have been covered by maps
# Contains lots of 3/4's (no map coverage), requires further investigation to see if any maps preset
df_no_evidence_no_zero = df_no_evidence[df_no_evidence.zero_combined ==0]
df_no_evidence_no_zero['Category'] = 'Maybe ancient woodland'
df_no_evidence_no_zero['subCategory'] = 'D'

## C - Not ancient woodland
## No evidence, and contains at least one zero, so assumed not woodland. 
## 1637, ~600 1 zero, ~1000 more than one zero
df_no_evidence_zero = df_no_evidence[df_no_evidence.zero_combined >= 1]
df_no_evidence_zero['Category'] = 'Not ancient woodland'
df_no_evidence_zero['subCategory'] = 'C'

df_no_evidence_one_zero = df_no_evidence[df_no_evidence.zero_combined == 1]
df_no_evidence_plural_zero = df_no_evidence[df_no_evidence.zero_combined > 1]



C:\Users\Lara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Lara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Lara\Ap

In [56]:
#Filter for secondary evidence present, and no zeros present
df_secondary_no_zero = df_secondary[df_secondary.zero_combined ==0]

## C - Ancient woodland 
## No primary evidence, but contains secondary evidence, no zero's present and is named. 
df_secondary_no_zero_named = df_secondary_no_zero[df_secondary_no_zero.name_present >=1]
df_secondary_no_zero_named['Category'] = 'Ancient woodland'
df_secondary_no_zero_named['subCategory'] = 'C'

## B- Maybe ancient woodland
## No primary evidence, but contains secondary evidence, no zero's present, but is not named
## Further research needed; why is not present on primary evidence? 
df_secondary_no_zero_not_named  = df_secondary_no_zero[df_secondary_no_zero.name_present ==0]
df_secondary_no_zero_not_named['Category'] = 'Maybe ancient woodland'
df_secondary_no_zero_not_named['subCategory'] = 'B'

#Filter for secondary evidence present, and one zero present
df_secondary_one_zero = df_secondary[df_secondary.zero_combined ==1]

## C - Maybe ancient woodland
## No primary evidence, secondary evidence, but one zero present, and named
## Further research; why no primary evidence? and what source is zero? What source is name? 
df_secondary_one_zero_named  = df_secondary_one_zero[df_secondary_one_zero.name_present >=1]
df_secondary_one_zero_named['Category'] = 'Maybe ancient woodland'
df_secondary_one_zero_named['subCategory'] = 'C'

## E - Maybe ancient woodland
## No primary evidence, secondary evidence, but one zero present, not named
## Probably not ancient woodland, but check primary evidence, and zero. 
df_secondary_one_zero_not_named  = df_secondary_one_zero[df_secondary_one_zero.name_present ==0]
df_secondary_one_zero_not_named['Category'] = 'Maybe ancient woodland'
df_secondary_one_zero_not_named['subCategory'] = 'E'

## B - Not ancient woodland
## Multiple zeros present, and no primary evidence so probably not ancient woodland. 
df_secondary_plural_zero = df_secondary[df_secondary.zero_combined >1]
df_secondary_plural_zero['Category'] = 'Not ancient woodland'
df_secondary_plural_zero['subCategory'] = 'B'

df_secondary_zero = df_secondary[df_secondary.zero_combined >=1]

C:\Users\Lara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Lara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [57]:


df_combined = pd.concat([df_primary_no_zero,
df_primary_plural_zero,
df_primary_one_zero_named,
df_primary_one_zero_not_named,
df_no_evidence_no_zero,
df_no_evidence_zero,
df_secondary_no_zero_named,
df_secondary_no_zero_not_named,
df_secondary_one_zero_named,
df_secondary_one_zero_not_named,
df_secondary_plural_zero,])

In [31]:
df_ancient_only = df_combined[df_combined.Category == 'Ancient woodland']
df_maybe_ancient = df_combined[df_combined.Category == 'Maybe ancient woodland']
df_not_ancient = df_combined[df_combined.Category == 'Not ancient woodland']

In [51]:
df_not_ancient_grouped = df_not_ancient.groupby('P3_Status').count().reset_index()
df_ancient_grouped = df_ancient_only.groupby('P3_Status').count().reset_index()

In [87]:
df_combined['area'] = df_combined.area/10000

In [88]:
df_grouped = df_combined.groupby(['Category', 'subCategory','intersects_old_AWI'])['area'].sum().reset_index()


In [67]:
df_intersects_not_ancient = df_combined[(df_combined['intersects_old_AWI']==True)&(df_combined['Category']=='Not ancient woodland')]
df_intersects_not_ancient.area.sum()/10000

np.float64(670.9962464495912)

In [90]:
df_grouped_not_ancient  = df_grouped[df_grouped.Category =='Not ancient woodland']

In [91]:
fig = px.bar(df_grouped_not_ancient, x ='subCategory', y = 'area' , color='intersects_old_AWI')
fig.show()

# Decision tree diagram

In [149]:
df_sankey = pd.read_csv('../sankey lookup.csv')

In [156]:


fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
     # line = dict(color = "black", width = 0.5),
      label =df_sankey['name2'],
      color = df_sankey['node_colour']
    ),
    link = dict(
      source = df_sankey['source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = df_sankey['target'],
      value = df_sankey['value'],
      color = df_sankey['colour']
  ))])

fig.update_layout(title_text="AWI Decision Tree", font_size=10)
fig.show()

In [183]:
gdf_old = gpd.read_file("C:/Users/Lara/AWI/Old_AWI_Wilts.shp")

In [182]:
gdf_new = gpd.read_file("C:/Users/Lara/AWI/New_AWI.shp")

In [184]:
gdf_new_reduced = gdf_new[['NAME_OS', 'NAME_EP1', 'P3_UID', 'P3_IGNORE', 'C21_AP', 'EPOCH_2',
       'EPOCH_1', 'C20_AP', 'C20_AP_REF', 'TITHE_REF', 'C19_EM', 'C19_EM_REF',
       'OSD', 'OSD_REF', 'CM_1773', 'C18_EM', 'C18_EM_REF', 'C17_EM',
       'C17_EM_REF', 'EM', 'EM_REF', 'OTHER', 'OTHER_REF', 'NAME_TITHE',
       'NAME_OSD', 'NAME_EM', 'TITHE_REF_', 'TITHE_LAND', 'TITHE_PLOT',
       'TITHE_PL_1', 'COMMENTS', 'url', 'OSNAMES_UR', 'CHECK_NAME',
       'TO_CHECK_N', 'NOTES', 'LISTED_BUI', 'LISTED_PAR', 'P2_CLASS',
       'P3_CLASS', 'P3_Status', 'TITHE', 'GlobalID_1', 'EARLIEST_E', 'RESERVE',
       'FORESTRY', 'SURVEY_PRI', 'ACTION', 'SITE_ID', 'AreaHa', 'NAME',
       'Shape_Leng', 'OBJECTID_1', 'HE_REF', 'geometry']]

In [173]:
gdf_intersect = gdf_new.intersection(gdf_new)

In [185]:
gdf_intersect = gdf_new.intersection(gdf_old)

C:\Users\Lara\AppData\Local\Temp\ipykernel_20364\1796647971.py:1: UserWarning:

The indices of the left and right GeoSeries' are not equal, and therefore they will be aligned (reordering and/or introducing missing values) before executing the operation. If this alignment is the desired behaviour, you can silence this warning by passing 'align=True'. If you don't want alignment and protect yourself of accidentally aligning, you can pass 'align=False'.



In [187]:
intersecting_polygons = gdf_new[gdf_new.geometry.intersects(gdf_old.union_all())]

C:\Users\Lara\AppData\Local\Temp\ipykernel_20364\553252838.py:1: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.



In [196]:
#gdf_new['intersections'] = gdf_new.geometry.apply(lambda x: gdf_old.geometry.intersects(x).any())
gdf_new['intersects_test'] = gdf_new.geometry.intersects(gdf_old.geometry.unary_union)

C:\Users\Lara\AppData\Local\Temp\ipykernel_20364\3665420064.py:2: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.



In [198]:
gdf_new[gdf_new.intersections == True]

,NAME_OS,NAME_EP1,P3_UID,P3_IGNORE,C21_AP,EPOCH_2,EPOCH_1,C20_AP,C20_AP_REF,TITHE_REF,...,EM_0,OTHER_0,Zero_combi,Shape_Le_1,Shape_Ar_1,Assess_sec,Assess_pri,geometry,intersects,intersections
3,None,None,WiltsAWI_01717_01,1,1,5,1,5,None,NA,...,1,1,10111111,187.637685,1703.422370,5,1,"POLYGON ((431839.962 169618.899, 431849.442 16...",True,True
27,None,None,WiltsAWI_03682_01,1,1,5,1,5,None,NA,...,1,1,11110111,445.810370,6215.419083,5,0,"POLYGON ((395607.23 153387.62, 395605.77 15337...",True,True
28,None,None,WiltsAWI_03682_02,1,1,5,1,5,None,East Coulston,...,1,1,11110111,950.433644,21111.010173,5,1,"POLYGON ((395512.72 153131.59, 395507.61 15312...",True,True
35,None,None,WiltsAWI_04555_02,1,1,5,1,5,None,Horningsham,...,1,1,01111111,312.653820,2690.907350,6,1,"POLYGON ((380087.4 141439.75, 380090.98 141444...",True,True
37,Bayley's Coppice,None,WiltsAWI_05079_01,1,1,5,1,5,None,Stourton,...,1,1,01111111,963.118317,20883.315250,6,1,"POLYGON ((376096.88 135693.26, 376106.58 13569...",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,Milton Wood,None,WiltsAWI_03336_01,0,1,5,1,5,None,Everleigh,...,1,1,11111111,1281.402876,41103.630062,6,0,"POLYGON ((419437.64 156588.92, 419438.13 15658...",True,True
7072,Oldhat Copse,None,WiltsAWI_03378_01,0,1,5,1,5,None,Everleigh,...,1,1,11111111,560.880236,16615.874770,6,0,"POLYGON ((420345.013 156732.333, 420343.55 156...",True,True
7083,None,None,WiltsAWI_07109_12,0,1,5,1,5,None,Berwick St John,...,1,1,11111111,1248.142114,45067.049179,6,0,"POLYGON ((396910.378 119755.347, 396897.322 11...",True,True
7084,None,None,WiltsAWI_07109_12,0,1,5,1,5,None,Berwick St John,...,1,1,11111111,877.835445,18591.279938,6,0,"POLYGON ((396845.901 120020.269, 396842.01 120...",True,True
